# Points à vérifier dans mon code: 

In [1]:
# Trouver un moyen d'uniformiser 'time_step_per_hour' pour toute les données, genre dénominateur commun etc.
dates_verif_obj = DatesVerifFeatureVect(dataset.df_dates, Weeks = args.W, Days = args.D, historical_len = args.H, step_ahead = args.step_ahead, time_step_per_hour = dataset.time_step_per_hour)
dataset.get_feature_vect(invalid_dates)
# ========== .... ==========

# Passage dans split Tensor: 
# >>>>  Consiste à Normalizer un input (Ici 'U') selon une certaine référence (ici train_input)
# >>>>  Puis de renovyer les U_train/U_valid/U_test 
# >>>>
# >>>>   Pour se faire, ils calculent d'abord les stats (min,max,mean,std) sur le tensor de référence train_input
# >>>>   Puis, à chacun des split U_Train/U_Valid/U_Test, normalisation selon les statistiques.


# ==== ATTENTION il faudra sans doute récupérer les stats simplement et pas normalizer la ref
# ==== Calcul inutile ici 


# ========= PERSONAL_INPUT =========
# >>>> 
# >>>> PB_1 : Il faut absolument pouvoir prendre en compte les invalid dates de tout le monde.
# >>>> 1. Il le faire en amont. i.e lorsqu'on load les données, on load aussi les invalid dates associées
# >>>> 2. Puis on fait l'union des invalid dates
# >>>> 3. Après ça on train/valid/test split normalize tout.
# >>>> 
# >>>> PB_2: Idem pour le coverage pour que ça corresponde à tout le monde.  
# >>>> 
# >>>> PB_3 : Génère des input de taille différentes selon l'historique demandé. 
# >>>> Cela ne va pas convenir si jamais je travail avec du calendar class ou autre.
# >>>> 1. On peut donc enregistrer l'ensemble des invalid-dates
# >>>> 2. Créer les séquences/input_tensor associées 
# >>>> 3. Pour chaque donnée, on ne prend peut être pas en compte les même historiques. 
# >>>>    Donc on peut faire une liste de date 'D' correspondant à l'intersection de toute les target (df_verif[-1] ...?)
# >>>> 4. Puis on extrait l'ensemble des sequences de chaque Data sur D 


NameError: name 'DatesVerifFeatureVect' is not defined

# Mini Test pour voir si tout le pre-processing fonctionne: 

In [1]:
from DL_class import InvalidDatesCleaner,DatesVerifFeatureVect
from dataset import TrainValidTest_Split_Normalize,PersonnalInput
from calendar_class import get_time_slots_labels
import os
import pandas as pd 
import torch
import pickle 
import numpy as np 

from paths import folder_path,file_name
from config import get_args
from utilities_DL import get_DataSet_and_invalid_dates, match_period_coverage_with_netmob


# ==================== Load Subway-in Dataset : ====================
# Load config
model_name = 'STGCN' #'CNN'
netmob = True
args = get_args(model_name)
#args = get_args(model_name = model_name,learn_graph_structure = True)  # MTGNN

# Modification : 
args.K_fold = 1
args.ray = False

In [5]:
# Load Init DataSet
coverage = match_period_coverage_with_netmob(file_name)
if not torch.cuda.is_available(): 
    coverage = coverage[:100]
    args.W = 0
    args.D = 0


dataset,invalid_dates = get_DataSet_and_invalid_dates(args.abs_path, folder_path,file_name,
                                                      args.W,args.D,args.H,args.step_ahead,
                                                      single_station = False,coverage_period = coverage)
# ==================== ........................ ====================


if torch.cuda.is_available():
    data_folder_path = '../../../data/' 
else:
    data_folder_path = '../../Data/'


# === INIT ====
save_folder = f"{data_folder_path}NetMob_tensor/"
netmob_data_folder_path = f"{data_folder_path}NetMob/"
step_south_north = 287  # Incremente by 287-ids when passing from south to north. 
epsilon=1000  #epsilon : radius, in meter (1000m) 
# W,H = 2*(epsilon//100 + 1), 2*(epsilon//100 + 1)

if not os.path.exists(save_folder):
    os.makedirs(save_folder)
# === .... ===


# ===== Load NetMob Data: =====
# NetMob Tensor : [T,N,C,H,W]
# dims : [0,-2,-1]  -> dimension for which we want to retrieve stats 
try :
    netmob_T = torch.stack([torch.load(f"{save_folder}station_{station}.pt") for station in ref_subway.COD_TRG])
    netmob_T = netmob_T.permute(1,0,*range(2, netmob_T.dim()))

except:
    netmob_T = torch.randn(dataset.length,40,2,22,22)
    print("Load des données NetMob .pt impossible. Création d'un random Tensor")

print('Init NetMob Dataset: ', netmob_T.size())
print('Number of Nan Value: ',torch.isnan(netmob_T).sum())
print('Total Number of Elements: ', netmob_T.numel(),'\n')
# ===== ....... =====


print('Tackling Subway Data:')
subway_ds = PersonnalInput(invalid_dates, tensor = dataset.raw_values, dates = dataset.df_dates,
                           time_step_per_hour = dataset.time_step_per_hour,Weeks = args.W, Days = args.D, historical_len = args.H,step_ahead = args.step_ahead)
subway_ds.preprocess(args.train_prop,args.valid_prop,args.test_prop,dims_agg=[0])

tensor_limits_keeper = subway_ds.tensor_limits_keeper

print('\nTackling NetMob Data: ')
NetMob_ds = PersonnalInput(invalid_dates, tensor = netmob_T, dates = dataset.df_dates,
                           time_step_per_hour = dataset.time_step_per_hour,Weeks = args.W, Days = args.D, historical_len = args.H,step_ahead = args.step_ahead)
NetMob_ds.preprocess(args.train_prop,args.valid_prop,args.test_prop,dims_agg=[0,3,4])


print('\nTackling Calendar Data')
time_slots_labels,dic_class2rpz,dic_rpz2class,nb_words_embedding = get_time_slots_labels(subway_ds)
calendar_tensor = time_slots_labels[args.calendar_class]

splitter = TrainValidTest_Split_Normalize(calendar_tensor,
                            first_train = tensor_limits_keeper.first_train_U, last_train= tensor_limits_keeper.last_train_U,
                            first_valid= tensor_limits_keeper.first_valid_U, last_valid = tensor_limits_keeper.last_valid_U,
                            first_test = tensor_limits_keeper.first_test_U, last_test = tensor_limits_keeper.last_test_U)

train_tensor_ds,valid_tensor_ds,test_tensor_ds = splitter.load_normalize_tensor_datasets(normalize = False)
calendar_U_train,calendar_U_valid,calendar_U_test = train_tensor_ds.tensor,valid_tensor_ds.tensor,test_tensor_ds.tensor


Time-step per hour: 4.0
coverage period: 2019-03-16 00:00:00 - 2019-03-17 00:45:00
Load des données NetMob .pt impossible. Création d'un random Tensor
Init NetMob Dataset:  torch.Size([100, 40, 2, 22, 22])
Number of Nan Value:  tensor(0)
Total Number of Elements:  3872000 

Tackling Subway Data:
Tackling reference for normalization

 Tackling Training Set
Values with issues:  0.000%
Regular Values that we have to set to 0:  0.000%

 Tackling Validation Set
Values with issues:  0.000%
Regular Values that we have to set to 0:  0.000%

 Tackling Testing Set
Values with issues:  0.000%
Regular Values that we have to set to 0:  0.000%
Tackling reference for normalization

 Tackling Training Set
Values with issues:  0.000%
Regular Values that we have to set to 0:  0.000%

 Tackling Validation Set
Values with issues:  0.000%
Regular Values that we have to set to 0:  0.000%

 Tackling Testing Set
Values with issues:  0.000%
Regular Values that we have to set to 0:  0.000%

U size:  torch.Size(

In [6]:
from loader import DictDataLoader
# Train, Valid, Test split : 
train_tuple = subway_ds.U_train, subway_ds.Utarget_train, dict(netmob = NetMob_ds.U_train,calendar = calendar_U_train) #, calendar = calendar[train_subset]
valid_tuple = subway_ds.U_valid, subway_ds.Utarget_valid, dict(netmob = NetMob_ds.U_valid, calendar = calendar_U_valid ) #, calendar = calendar[train_subset]
test_tuple = subway_ds.U_test, subway_ds.Utarget_test, dict(netmob = NetMob_ds.U_test, calendar = calendar_U_test) #, calendar = calendar[train_subset]

# Load DictDataLoader: 
DictDataLoader_object = DictDataLoader(train_tuple, valid_tuple, test_tuple, args)
dict_dataloader = DictDataLoader_object.get_dictdataloader()

X_subway_b, Y_subway_b, contextual_data_b = next(iter(dict_dataloader['train']))
X_netmob_b = contextual_data_b[0]
X_calendar_b = contextual_data_b[1]

print(X_subway_b.size(),Y_subway_b.size(),X_netmob_b.size(),X_calendar_b.size())

torch.Size([24, 40, 6]) torch.Size([24, 40, 1]) torch.Size([24, 40, 2, 22, 22, 6]) torch.Size([24])


## Essais avec NetMob

In [4]:
T = netmob_T.size(0)

# Tackle a specific fold : 
netmob_T1 = netmob_T[:100]

# Init :
dims = [0,3,4]#[0,-2,-1]
minmaxnorm = True
standardize = False

# ============ Load Train/Valid/Test Indices and removed forbidden dates : ============
# invalid dates = 
# invalid_indices = get_indices_from_dates(invalid)
indices = np.arange(T)
np.random.shuffle(indices)
invalid_indices = indices[:100]

# Get Split indices :
train_indices = np.arange(50)
valid_indices = np.arange(60,70)
test_indices = np.arange(80,100)

# Remove invalid_dates from indices :
cleaner = InvalidDatesCleaner(invalid_indices = invalid_indices)

# >>>> Le cleaner prend un ensemble d'indice qui est interdit (les indices correspondant au dates qui sont interdites)
# >>>> On lui passe ensemble un ensemble d'indices (ex : ceux qui correspondent à l'extraction de U_train)
# >>>> Pour lequels il va masqué/supprimer les indices interdit
# >>>> On ce sert ensuite de cet ensemble pour extraire le bon U_train, avec les valeurs qui sont effectivement interdite. 


train_indices = cleaner.clean_indices(train_indices)
valid_indices = cleaner.clean_indices(valid_indices)
test_indices = cleaner.clean_indices(test_indices)
# ============ .......................................................... ============

# Load Splitter Object
splitter = TrainValidTest_Split_Normalize(netmob_T1,dims,train_indices, valid_indices, test_indices,minmaxnorm=minmaxnorm,standardize=standardize)

# Split DataSet and Normalize accoding Stats from Training Set 
train_dataset,valid_dataset,test_dataset = splitter.load_normalize_tensor_datasets()
# Define DictDataLoader :


 Tackling Training Set


IndexError: min(): Expected reduction dim 2 to have non-zero size.